# Make predictions and get results

## Predict on test set

In [1]:
from allennlp.models.archival import archive_model
import pickle
import numpy as np
from allennlp.models import Model

/home/del/anaconda3/envs/allennmt/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#archive_model("models/tanh_2l_ff.json/")

In [3]:
from allennlp.models.archival import load_archive
from allennlp.service.predictors import Predictor

# required so that our custom model + predictor + dataset reader
# will be registered by name
import bigram_embedder

# with open("data/step5_holduot_splits/train_all_shuffled.pkl", 'rb') as f:
#     train_zip = pickle.load(f)
# with open("data/step5_holduot_splits/dev_all_shuffled.pkl", 'rb') as f:
#     dev_zip = pickle.load(f)
with open("data/step5_holduot_splits/test_all_shuffled.pkl", 'rb') as f:
    test_zip = pickle.load(f)
    

#archive = load_archive('models/multi_tanh_300/model.tar.gz')
#archive = load_archive('models/tanh_2l/model.tar.gz')
archive = load_archive('models/4l_100/model.tar.gz')
predictor = Predictor.from_archive(archive, 'bigram-embedder')


In [4]:
with open("data/step5_holduot_splits/test_all_shuffled.tsv") as f:
    X_test = f.read().split('\n')

In [5]:
bigram_types_test = []
bigram_vecs_test = []
for l in X_test:
    bt, bv, _, _ = l.split('\t')
    bigram_types_test.append(bt)
    bigram_vecs_test.append([float(v) for v in bv.split()])

In [6]:
prediction_method = "model"
#prediction_method = "sum"
#prediction_method = "minus"
#prediction_method = "mul"


In [7]:
from operator import add
from operator import sub
from operator import mul
# summing up / avaraging

if prediction_method == "sum":
    bigram_vecs_test_hat = []
    for ex in X_test:
        ex = predictor.load_line(ex)
        w1_vec = [float(i) for i in ex['w1_vec_str'].split()]
        w2_vec = [float(i) for i in ex['w2_vec_str'].split()]
        res_vec = list(map(add, w1_vec, w2_vec))
        bigram_vecs_test_hat.append(res_vec)
        #bigram_vecs_test_hat.append(predictor.predict_json(ex).get("bigram_vecs_hat"))
        # OR model
elif prediction_method == "minus":
    bigram_vecs_test_hat = []
    for ex in X_test:
        ex = predictor.load_line(ex)
        w1_vec = [float(i) for i in ex['w1_vec_str'].split()]
        w2_vec = [float(i) for i in ex['w2_vec_str'].split()]
        res_vec = list(map(sub, w1_vec, w2_vec))
        bigram_vecs_test_hat.append(res_vec)
        #bigram_vecs_test_hat.append(predictor.predict_json(ex).get("bigram_vecs_hat"))
        # OR model
elif prediction_method == "mul":
    bigram_vecs_test_hat = []
    for ex in X_test:
        ex = predictor.load_line(ex)
        w1_vec = [float(i) for i in ex['w1_vec_str'].split()]
        w2_vec = [float(i) for i in ex['w2_vec_str'].split()]
        res_vec = list(map(mul, w1_vec, w2_vec))
        bigram_vecs_test_hat.append(res_vec)
        #bigram_vecs_test_hat.append(predictor.predict_json(ex).get("bigram_vecs_hat"))
        # OR model
elif prediction_method == "model":
    bigram_vecs_test_hat = []
    for ex in X_test:
        ex = predictor.load_line(ex)
        bigram_vecs_test_hat.append(predictor.predict_json(ex).get("bigram_vecs_hat"))

## Compute error

In [8]:
import torch

In [9]:
def compute_smooth_l1_error(y, y_hat):
    y_var = torch.autograd.Variable(torch.FloatTensor(y))
    y_hat_var = torch.autograd.Variable(torch.FloatTensor(y_hat))
    
    if y_var.size() != y_hat_var.size():
        smooth_l1_error = torch.nn.SmoothL1Loss(reduce=False)
        y_var = y_var.expand(y_hat_var.size())
        return smooth_l1_error.forward(y_var, y_hat_var).sum(1).data

    else:
        smooth_l1_error = torch.nn.SmoothL1Loss()
        return smooth_l1_error.forward(y_var, y_hat_var).data[0]    

## Comput accuracy

In [10]:
def get_closest(target, vecs, n):
    sim_matrix = compute_smooth_l1_error(target, vecs)
    nvecs, nids = sim_matrix.sort(0)
    return nvecs[0:n], nids[0:n]

In [11]:
# vars
PATH_DATA_STEP3_FOLDER = "data/step3_fasttext_vectors/"
PATH_FASTTEXT_VECTORS = PATH_DATA_STEP3_FOLDER + "vectors_likelihood_ratio-100-0.05.vec"
SUFFIX = PATH_FASTTEXT_VECTORS.split('vectors_')[1][0:-4]
PATH_DATA_OUTPUT_FOLDER = "data/step4_wordpairs_bigrams_vec_data/" 

In [12]:
def get_all_ngram_vecs():
    # load fasttext .vec file containing unigrams and bigrams
    with open(PATH_FASTTEXT_VECTORS, 'r') as f:
        vectors_all = f.readlines()
        
    num_vectors_all, dim = vectors_all[0].split()
    num_vectors_all, dim = int(num_vectors_all), int(dim)
    
    print(num_vectors_all)
    print(dim)
    
    del vectors_all[0]
    
    ngram_types_all = []
    ngram_vectors_all = []
    for l in vectors_all:
        l = l.split()
        ngram_types_all.append(l[0])
        ngram_vectors_all.append([float(v) for v in l[1:]])
    return ngram_types_all, ngram_vectors_all

In [13]:
%%time
#all_bigrams, all_vecs, _ = zip(*train_zip + dev_zip + test_zip)
all_bigrams, all_vecs = get_all_ngram_vecs()

808968
100
CPU times: user 29.8 s, sys: 1.43 s, total: 31.3 s
Wall time: 31.7 s


In [ ]:
%%time
NUM_NEIGHBOURS = 100
nearest_vecs_test = []
nearest_ids_test = []
for i, vec_hat in enumerate(bigram_vecs_test_hat):
    nvecs, nids = get_closest(vec_hat, all_vecs, NUM_NEIGHBOURS)
    nearest_vecs_test.append(nvecs)
    nearest_ids_test.append(nids)
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [15]:
predicted_bigrams = []
for i, ids in enumerate(nearest_ids_test):
    predicted_bigram_list = [all_bigrams[i] for i in ids]
    predicted_bigrams.append(predicted_bigram_list)

In [16]:
Ns = [1,3,5,7,10, 100]
for N in Ns:
    corrects = []
    corrects_groundtruth = []
    errors = []
    errors_groundtruth = [] 
    for i, predicted_bigrams_list in enumerate(predicted_bigrams):
        if bigram_types_test[i] in predicted_bigrams_list[0:N]:
            corrects.append(predicted_bigrams_list)
            corrects_groundtruth.append(bigram_types_test[i])
        else:
            errors.append(predicted_bigrams_list)
            errors_groundtruth.append(bigram_types_test[i])

    print('Top', N, 'accuaracy on test set:', len(corrects) / len(predicted_bigrams) * 100, '%')

Top 1 accuaracy on test set: 70.88 %
Top 3 accuaracy on test set: 84.76 %
Top 5 accuaracy on test set: 87.76 %
Top 7 accuaracy on test set: 90.0 %
Top 10 accuaracy on test set: 91.72 %
Top 100 accuaracy on test set: 97.0 %


In [16]:
Ns = [1,3,5,7,10, 100]
for N in Ns:
    corrects = []
    corrects_groundtruth = []
    errors = []
    errors_groundtruth = [] 
    for i, predicted_bigrams_list in enumerate(predicted_bigrams):
        if bigram_types_test[i] in predicted_bigrams_list[0:N]:
            corrects.append(predicted_bigrams_list)
            corrects_groundtruth.append(bigram_types_test[i])
        else:
            errors.append(predicted_bigrams_list)
            errors_groundtruth.append(bigram_types_test[i])

    print('Top', N, 'accuaracy on test set:', len(corrects) / len(predicted_bigrams) * 100, '%')

Top 1 accuaracy on test set: 62.03999999999999 %
Top 3 accuaracy on test set: 74.96000000000001 %
Top 5 accuaracy on test set: 79.75999999999999 %
Top 7 accuaracy on test set: 82.16 %
Top 10 accuaracy on test set: 84.68 %
Top 100 accuaracy on test set: 93.12 %


## Error analysis

In [17]:
import random

In [18]:
n = 10

In [19]:
random_n_ids = [random.randrange(0, len(errors)) for i in range(n)]

for ind in random_n_ids:
    ngrams = errors[ind]
    print(ind, '|', errors_groundtruth[ind], '->', ngrams[0:5])

88 | of_rf -> ['statvolt', 'sincgars', 'qmv', 'currentfib', 'milliamperes']
152 | ecw_s -> ['wwe_and', 'and_wcw', 'comeback_the', 'thefutureembrace', 'the_smackdown']
169 | v_has -> ['however_has', 'is_likewise', 'and_furthermore', 'called_has', 'furthermore_since']
84 | s_perhaps -> ['perhaps', 'perhaps_ironically', 'perhaps_surprisingly', 'perhaps_just', 'however_perhaps']
74 | be_laid -> ['be_abandoned', 'laid_for', 'laid_and', 'being_laid', 'be_cleared']
7 | as_pm -> ['pm', 'bqe', 'frequently_one', 'frequently_observed', 'as_show']
165 | her_play -> ['playing_herself', 'mother_played', 'on_herself', 'janet_and', 'a_backstage']
58 | met_in -> ['met', 'finally_met', 'attended_with', 'recalled_from', 'also_in']
132 | iv_v -> ['iv', 'iii', 'v', 'vii', 'iii_this']
82 | lb_or -> ['extra_weight', 'additional_weight', 'normal_weight', 'weight_five', 'zero_kcal']


In [20]:
whos

Variable                  Type                          Data/Info
-----------------------------------------------------------------
Model                     type                          <class 'allennlp.models.model.Model'>
N                         int                           100
NUM_NEIGHBOURS            int                           100
Ns                        list                          n=6
PATH_DATA_OUTPUT_FOLDER   str                           data/step4_wordpairs_bigrams_vec_data/
PATH_DATA_STEP3_FOLDER    str                           data/step3_fasttext_vectors/
PATH_FASTTEXT_VECTORS     str                           data/step3_fasttext_vecto<...>lihood_ratio-100-0.05.vec
Predictor                 type                          <class 'allennlp.service.<...>ors.predictor.Predictor'>
SUFFIX                    str                           likelihood_ratio-100-0.05
X_test                    list                          n=2500
add                       builtin_function_or

In [21]:
# with open('data/nearest_vecs_test' + SUFFIX + '.pkl', 'wb') as f:
#     pickle.dump(nearest_vecs_test, f)
    
# with open('data/nearest_ids_test' + SUFFIX + '.pkl', 'wb') as f:
#     pickle.dump(nearest_ids_test, f)

## Make predictions

In [22]:
a = [1,2,3]

In [23]:
def pred_example(w1, w2, N=5):
    vec1 = all_vecs[all_bigrams.index(w1)]
    vec2 = all_vecs[all_bigrams.index(w2)]
    x = {'w1_vec_str': ' '.join([str(v) for v in vec1]), 'w2_vec_str': ' '.join([str(v) for v in vec2])}
    bi_vec = predictor.predict_json(x)['bigram_vecs_hat']
    _, ids_hat = get_closest(bi_vec, all_vecs, N)
    bigrams_hat = [all_bigrams[i] for i in ids_hat]
    return bigrams_hat

In [24]:
pair = 'this album'
w1, w2 = pair.split(' ')
pred_example(w1, w2)

['another_album',
 'album_another',
 'following_album',
 'album_itself',
 'this_album']

In [25]:
pair = 'contemporary methods'
w1, w2 = pair.split(' ')
pred_example(w1, w2)

['modern_methods',
 'traditional_methods',
 'conventional_methods',
 'historical_method',
 'modern_techniques']

In [26]:
pair = 'rephrase me'
w1, w2 = pair.split(' ')
pred_example(w1, w2)

['rephrase', 'you_something', 'damn_thing', 'you_simply', 'says_you']

In [27]:
pair = 'say something'
w1, w2 = pair.split(' ')
pred_example(w1, w2)

['say_something',
 'know_something',
 'that_something',
 'us_something',
 'says_something']

In [28]:
pair = 'domain name'
w1, w2 = pair.split(' ')
pred_example(w1, w2)

['name', 'three_domain', 'appropriate_name', 'name_now', 'temporary_name']

In [29]:
pair = 'thug live'
w1, w2 = pair.split(' ')
pred_example(w1, w2)

['dirty_bastard', 'crazy_little', 'screaming_in', 'stray_dog', 'pretty_hate']

In [30]:
pair = 'state_of the_art'
w1, w2 = pair.split(' ')
pred_example(w1, w2)

['trance_state',
 'the_art',
 'great_state',
 'state_history',
 'general_understanding']

In [31]:
pair = 'state_of art'
w1, w2 = pair.split(' ')
pred_example(w1, w2)

['trance_state',
 'art',
 'western_art',
 'contemporary_culture',
 'intellectual_history']

In [32]:
pair = 'state of_art'
w1, w2 = pair.split(' ')
pred_example(w1, w2)

['of_art',
 'western_art',
 'national_art',
 'cultural_institutions',
 'that_institution']

In [33]:
def substract_add(n1, n2, n3, N=5):
    try:
        vec1 = np.array(all_vecs[all_bigrams.index(n1)])
    except(ValueError):
        print('OOV ngram; estimating vec for it')
        vec1 = pred_example(n1.split('_')[0], n1.split('_')[1], return_vec=True)
        
    vec2 = np.array(all_vecs[all_bigrams.index(n2)])
    vec3 = np.array(all_vecs[all_bigrams.index(n3)])
    #x = {'w1_vec_str': ' '.join([str(v) for v in vec1]), 'w2_vec_str': ' '.join([str(v) for v in vec2])}
    v_res = vec1 - vec2 + vec3
    # bi_vec = predictor.predict_json(x)['bigram_vecs_hat']
    _, ids_hat = get_closest(v_res, all_vecs, N)
    bigrams_hat = [all_bigrams[i] for i in ids_hat]
    return bigrams_hat

In [34]:
substract_add('go_to', 'go', 'run')

['run', 'running_for', 'also_running', 'that_running', 'at_running']